Import Dependencies

In [1]:
! conda list

# packages in environment at C:\Users\mebader.NORTHAMERICA\AppData\Local\Continuum\anaconda3\envs\PythonData:
#
# Name                    Version                   Build  Channel
aniso8601                 7.0.0                      py_0    conda-forge
antiorm                   1.2.1                    pypi_0    pypi
anyjson                   0.3.3                    py37_1  
argon2_cffi               19.1.0           py37he774522_0  
arrow                     0.13.1                   py37_0  
asn1crypto                0.24.0                   py37_0  
astroid                   2.2.5                    py37_0  
attrs                     19.1.0                   pypi_0    pypi
babel                     2.7.0                      py_0  
backcall                  0.1.0                    pypi_0    pypi
bcrypt                    3.1.6            py37he774522_0  
beautifulsoup4            4.8.0                    pypi_0    pypi
blas                      1.0                         mkl  
blea

In [3]:
import pandas as pd
import os


In [4]:
from bs4 import BeautifulSoup as bs

In [5]:
from selenium import webdriver  

In [6]:
from splinter import Browser

driver = webdriver.Chrome(executable_path=os.path.abspath("chromedriver.exe"))
                                   

In [7]:
import requests
import html5lib

Open URLs and Get Info

In [8]:
#Mars News portion
mars_news = {}
mars_para = []

In [9]:
base_url = "https://mars.nasa.gov/" 
news_url = "https://mars.nasa.gov/news"
resp_1 = requests.get(news_url)                                             

mars_soup = bs(resp_1.text, 'html.parser') 

In [10]:
class_div = mars_soup.find(class_="slide")                                  
marssoup_news = class_div.find_all('a')                                           
marsnews_title = marssoup_news[1].get_text().strip()  

In [11]:
soup_p = class_div.find_all('a', href=True)                                 
soup_p_url = soup_p[0]['href']                                               
paragraph_url = base_url + soup_p_url                                        
response_2 = requests.get(paragraph_url)                                          
paragraph_soup = bs(response_2.text, "html.parser")                               
mars_paragraphs = paragraph_soup.find(class_='wysiwyg_content')                     
paragraphs = mars_paragraphs.find_all('p')

In [12]:
for paragraph in paragraphs:                                                 
    simple_paragraph = paragraph.get_text().strip()                              
    mars_para.append(simple_paragraph)   

In [13]:
mars_news["news_title"] = marsnews_title
mars_news["paragraph_1"] = mars_para[0]
mars_news["paragraph_2"] = mars_para[1]

In [14]:
mars_news


{'news_title': 'NASA Invites Students to Name Mars 2020 Rover',
 'paragraph_1': 'Red rover, red rover, send a name for Mars 2020 right over! NASA is recruiting help from students nationwide to find a name for its next Mars rover mission.',
 'paragraph_2': 'Starting Tuesday, Aug. 27, K-12 students in U.S. public, private and home schools can enter the Mars 2020 Name the Rover essay contest. One grand prize winner will name the rover and be invited to see the spacecraft launch in July 2020 from Cape Canaveral Air Force Station in Florida.'}

Now get images from the jpl site to bring in to the web page

In [15]:
browser = Browser('chrome', headless=False) 
#browser = browser('Chrome')
jpl_base_url = 'https://photojournal.jpl.nasa.gov/jpeg/'               
jpl_image_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"

In [16]:
browser.visit(jpl_image_url)
image_html = browser.html
image_soup = bs(image_html, 'html.parser')

In [17]:
featured_image_list = []
for image in image_soup.find_all('div',class_="img"):
    featured_image_list.append(image.find('img').get('src'))

In [18]:
picture_shown = featured_image_list[0]
remove_size = picture_shown.split('-')
remove_fn = remove_size[0].split('/')
featured_image_url = jpl_base_url + remove_fn[-1] + '.jpg'


In [19]:
featured_image_url

'https://photojournal.jpl.nasa.gov/jpeg/PIA23449.jpg'

In [20]:
browser.quit()

Get Mars Twitter feed

In [21]:
browser = Browser('chrome', headless=False)
twitter_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(twitter_url)

In [22]:
twitter_html = browser.html
twitter_soup = bs(twitter_html, 'html.parser')

In [23]:
#twitter_soup.find('div', attrs={"class": "tweet", "data-name": "Mars Weather"})
mars_weather = twitter_soup.find("p", class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text").text
print(mars_weather)


#mars_weather

Weeeeee!  https://www.jpl.nasa.gov/spaceimages/details.php?id=PIA23461 …pic.twitter.com/iuEqJjsQ4G


Get Mars Facts

In [24]:
mars_facts_url = 'https://space-facts.com/mars/'


In [25]:
mars_facts_pd = pd.read_html(mars_facts_url)

In [26]:
mars_facts_df = mars_facts_pd[0]

In [27]:
mars_facts_table = mars_facts_df.to_html(header=False, index=False)
print(mars_facts_table)

<table border="1" class="dataframe">
  <tbody>
    <tr>
      <td>Diameter:</td>
      <td>6,779 km</td>
      <td>12,742 km</td>
    </tr>
    <tr>
      <td>Mass:</td>
      <td>6.39 × 10^23 kg</td>
      <td>5.97 × 10^24 kg</td>
    </tr>
    <tr>
      <td>Moons:</td>
      <td>2</td>
      <td>1</td>
    </tr>
    <tr>
      <td>Distance from Sun:</td>
      <td>227,943,824 km</td>
      <td>149,598,262 km</td>
    </tr>
    <tr>
      <td>Length of Year:</td>
      <td>687 Earth days</td>
      <td>365.24 days</td>
    </tr>
    <tr>
      <td>Temperature:</td>
      <td>-153 to 20 °C</td>
      <td>-88 to 58°C</td>
    </tr>
  </tbody>
</table>


Get Mars Hemispheres photos

In [28]:
import time 
hemispheres_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hemispheres_url)
html = browser.html
soup = bs(html, "html.parser")
mars_hemisphere = []

products = soup.find("div", class_ = "result-list" )
hemispheres = products.find_all("div", class_="item")

for hemisphere in hemispheres:
    title = hemisphere.find("h3").text
    title = title.replace("Enhanced", "")
    end_link = hemisphere.find("a")["href"]
    image_link = "https://astrogeology.usgs.gov/" + end_link    
    browser.visit(image_link)
    html = browser.html
    soup= bs(html, "html.parser")
    downloads = soup.find("div", class_="downloads")
    image_url = downloads.find("a")["href"]
    mars_hemisphere.append({"title": title, "img_url": image_url})

In [29]:
mars_hemisphere

[{'title': 'Cerberus Hemisphere ',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere ',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere ',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere ',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]